# 爬中文文献

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import csv
import os

# 设置当前工作目录
os.chdir('D:\\漂绿文献\\爬虫\\文献爬虫数据') # 设置当前工作目录为你要保存文件的目录
keyword='esg' # 输入检索词
# 初始化WebDriver
driver = webdriver.Edge()
url = 'https://kns.cnki.net/kns8s/AdvSearch?crossids=YSTT4HG0%2CLSTPFY1C%2CJUP3MUPD%2CMPMFIG1A%2CWQ0UVIAA%2CBLZOG7CK%2CPWFIRAGL%2CEMRPGLPA%2CNLBO1Z6R%2CNN3FJMUV'
driver.get(url)
time.sleep(2)


# 点击知网检索输入栏，输入你的检索词
driver.find_element(By.XPATH, '//*[@id="gradetxt"]/dd[1]/div[2]/input').click()
driver.find_element(By.XPATH,'//*[@id="gradetxt"]/dd[1]/div[2]/input').send_keys(keyword) #检索词
time.sleep(2)

# 进行检索//*                     
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[2]/div[3]/input').click()
time.sleep(4)
# 显示详细信息，为了爬取摘要、关键词
#driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[2]/div/div[1]/div/div[2]/ul[2]/li[1]/i').click()
#time.sleep(2)


# 每页显示条数改成50
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/div/i').click()
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/ul/li[3]/a').click()
time.sleep(3)


#只获取学术论文，排除学位论文、会议论文、报纸等
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[2]/div/div/ul/li[1]').click()
time.sleep(2)
# 点击“学术论文”

# 创建空列表以存储所有元素的文本内容
column1_texts = []
column2_texts = []
column3_texts = []
column4_texts = []
column5_texts = []
column6_texts = []
column7_texts = []
a=1
# 循环遍历每一页
while True:
    # 获取当前页的元素数量  //*[@id="gridTable"]/div/div/div/table/tbody
    elements_count = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/table/tbody/tr'))
    if elements_count != 50:
        print(f"当前页{a}元素数量: {elements_count}，不是50，可能是最后一页")
    # 遍历当前页的元素，获取文本内容并存储到列表中
    for i in range(1, elements_count + 1):
        # 第一列 文章题目      //*[@id="gridTable"]/div/div/div/table/tbody/tr[1]/td[2]/a
        try:
            element_col1 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[2]/a')
            column1_texts.append(element_col1.text)
            column7_texts.append(element_col1.get_attribute('href')) # 获取链接
        except NoSuchElementException:
            column1_texts.append("NA")

        # 第二列 作者      
        try:
            element_col2 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[3]')
            column2_texts.append(element_col2.text)
        except NoSuchElementException:
            column2_texts.append("NA")

        # 第三列 期刊        
        try:
            element_col3 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[4]/span/a')
            column3_texts.append(element_col3.text)
        except NoSuchElementException:
            column3_texts.append("NA")

        # 第四列 时间        
        try:
            element_col4 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[5]')
            column4_texts.append(element_col4.text)
        except NoSuchElementException:
            column4_texts.append("NA")

        # 第五列 被引      
        try:
            element_col5 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[6]')
            column5_texts.append(element_col5.text)
        except NoSuchElementException:
            column5_texts.append("NA")

        # 第六列 下载       
        try:
            element_col6 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[7]')
            column6_texts.append(element_col6.text)
        except NoSuchElementException:
            column6_texts.append("NA")



    # 尝试点击下一页按钮
    try:
        driver.find_element(By.ID, 'PageNext').click()
        print(f'正在爬一轮第{a}页')
        a += 1
        time.sleep(3)  # 等待页面加载 
    except NoSuchElementException:
        break  # 如果找不到下一页按钮，跳出循环 
    
df1=pd.DataFrame({
    '题名': column1_texts,
    '链接': column7_texts,
    '作者': column2_texts,
    '期刊名称': column3_texts,
    '时间': column4_texts,
    '被引': column5_texts,
    '下载': column6_texts
    })


driver.find_element(By.ID, 'total').click() #点击首页
time.sleep(2)  # 等待页面加载

# 显示详细信息，为了爬取摘要、关键词
driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[2]/div/div[1]/div/div[2]/ul[2]/li[1]/i').click()
time.sleep(2)



#爬摘要和关键词
title_texts = []
abstract_texts = []
keywords_texts = []
a = 1 # 页码计数器

while True:
    elements_count = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/dl/dd'))
    if elements_count !=50:
        try:
            elements_count_contain = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/dl/font/dd'))
            elements_count = elements_count_contain + elements_count
            print(f"当前页{a}的1叠元素数量: {elements_count}")
            if elements_count != 50:
                elements_count_contain2 = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/dl/font/font/dd'))
                elements_count = elements_count_contain2 + elements_count
                print(f"当前页{a}的2叠元素数量: {elements_count}")
        except NoSuchElementException:
            elements_count = elements_count
            print(f"当前页{a}的元素数量: {elements_count}，不是50个，可能是最后一页")

    b=c=1
    for i in range(1, elements_count + 1):
        #题目路径
        xpath1=f'//*[@id="gridTable"]/div/div/div/dl/dd[{i}]/div[2]/h6/a'
        xpath2=f'//*[@id="gridTable"]/div/div/div/dl/font/dd[{b}]/div[2]/h6/a'
        xpath3=f'//*[@id="gridTable"]/div/div/div/dl/font/font/dd[{c}]/div[2]/h6/a'
        #摘要路径 
        xpath4=f'//*[@id="gridTable"]/div/div/div/dl/dd[{i}]/div[2]/p[2]/span[2]'
        xpath5=f'//*[@id="gridTable"]/div/div/div/dl/font/dd[{b}]/div[2]/p[2]/span[2]'
        xpath6=f'//*[@id="gridTable"]/div/div/div/dl/font/font/dd[{c}]/div[2]/p[2]/span[2]'
        #关键词路径 
        xpath7=f'//*[@id="gridTable"]/div/div/div/dl/dd[{i}]/div[2]/p[3]'
        xpath8=f'//*[@id="gridTable"]/div/div/div/dl/font/dd[{b}]/div[2]/p[3]'
        xpath9=f'//*[@id="gridTable"]/div/div/div/dl/font/font/dd[{c}]/div[2]/p[3]'
        #获取题目
        try:
            title_text = driver.find_element(By.XPATH, xpath1) 
            title_texts.append(title_text.text)
        except NoSuchElementException:
            try:
                title_text = driver.find_element(By.XPATH, xpath2) 
                title_texts.append(title_text.text)
                b+=1 
            except:
                try:
                    title_text = driver.find_element(By.XPATH,xpath3)
                    title_texts.append(title_text.text)
                    c+=1
                except:
                    title_texts.append("NA")
                    print(f"当前页{a}的元素数量: {elements_count}，第{i}个元素没有找到标题")
            
        #获取摘要
        try:
            abstract_text = driver.find_element(By.XPATH, xpath4) 
            abstract_texts.append(abstract_text.text)
        except NoSuchElementException:
            try:
                abstract_text = driver.find_element(By.XPATH, xpath5) 
                abstract_texts.append(abstract_text.text)
            except:
                try:
                    abstract_text = driver.find_element(By.XPATH,xpath6)
                    abstract_texts.append(abstract_text.text)
                except:
                    abstract_texts.append("NA")
                    print(f"当前页{a}的元素数量: {elements_count}，第{i}个元素没有找到摘要")
        
        #获取关键词
        try:
            keywords_text = driver.find_element(By.XPATH, xpath7) 
            keywords_texts.append(keywords_text.text)
        except NoSuchElementException:
            try:
                keywords_text = driver.find_element(By.XPATH, xpath8) 
                keywords_texts.append(keywords_text.text)
            except:
                try:
                    keywords_text = driver.find_element(By.XPATH,xpath9)
                    keywords_texts.append(keywords_text.text)
                except:
                    keywords_texts.append("NA")
                    print(f"当前页{a}的元素数量: {elements_count}，第{i}个元素没有找到关键词")
    print(f'正在爬第{a}页')
    
    # 尝试点击下一页按钮
    try:
        driver.find_element(By.ID, 'PageNext').click()
        a += 1
        time.sleep(3)  # 等待页面加载 
    except NoSuchElementException:
        break  # 如果找不到下一页按钮，跳出循环
keywords_texts = [text.replace("关键词：", "") for text in keywords_texts] # 去掉关键词前的“关键词：”

df2=pd.DataFrame({
    '题名': title_texts,
    '摘要': abstract_texts,
    '关键词': keywords_texts
})
driver.quit()# 关闭浏览器

#合并数据
df3 = pd.merge(df1, df2, on='题名', how='left')


#读期刊目录
excel_df = pd.read_excel(r'D:\东财中文期刊分类.xlsx')
excel_df = excel_df[['期刊名称', '级别']]

#为期刊配级
df4= pd.merge(df3, excel_df, on='期刊名称', how='left')
from datetime import datetime
df4 = df4[['级别', '题名','链接', '作者', '期刊名称', '时间', '被引', '下载', '摘要', '关键词']]
df4['级别'] = df4['级别'].fillna('其他') # 将缺失值填充为'其他'

today_date = datetime.now().strftime('%Y-%m-%d')
file_excel = f'{keyword}_中文文献标级_{today_date}.xlsx'
df4.to_excel(file_excel, index=False)

# 爬外文文献

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import csv
import os

# 设置当前工作目录
os.chdir('D:\\漂绿文献\\爬虫\\文献爬虫数据') # 设置当前工作目录为你要保存文件的目录

# 初始化WebDriver
driver = webdriver.Edge()
url = 'https://kns.cnki.net/kns8s/AdvSearch?crossids=YSTT4HG0%2CLSTPFY1C%2CJUP3MUPD%2CMPMFIG1A%2CWQ0UVIAA%2CBLZOG7CK%2CPWFIRAGL%2CEMRPGLPA%2CNLBO1Z6R%2CNN3FJMUV'
driver.get(url)
time.sleep(2)

keyword='漂绿' 
# 点击知网检索输入栏，输入你的检索词
driver.find_element(By.XPATH, '//*[@id="gradetxt"]/dd[1]/div[2]/input').click()
driver.find_element(By.XPATH,'//*[@id="gradetxt"]/dd[1]/div[2]/input').send_keys(keyword) #检索词
time.sleep(3)

# 进行检索
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[2]/div[3]/input').click()
time.sleep(4)
# 每页显示条数改成50
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/div/i').click()
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/ul/li[3]/a').click()
time.sleep(3)


#只获取学术论文，排除学位论文、会议论文、报纸等
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[2]/div/div/ul/li[1]').click()
time.sleep(2)

# 点外文
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[2]/div/div/div/div/a[2]').click()
time.sleep(2)

# 创建空列表以存储所有元素的文本内容
column1_texts = []
column2_texts = []
column3_texts = []
column4_texts = []
column5_texts = []

a=1
# 循环遍历每一页
while True:
    # 获取当前页的元素数量  //*[@id="gridTable"]/div/div/div/table/tbody
    elements_count = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/table/tbody/tr'))
    if elements_count != 50:
        print(f"当前页{a}元素数量: {elements_count}，不是50，可能是最后一页")
    # 遍历当前页的元素，获取文本内容并存储到列表中
    for i in range(1, elements_count + 1):
        # 第一列 文章题目    
        try:
            element_col1 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[2]/a')
            column1_texts.append(element_col1.text)
            column5_texts.append(element_col1.get_attribute('href')) # 获取链接
        except NoSuchElementException:
            column1_texts.append("NA")

        # 第二列 作者      
        try:
            element_col2 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[3]')
            column2_texts.append(element_col2.text)
        except NoSuchElementException:
            column2_texts.append("NA")

        # 第三列 期刊        
        try:
            element_col3 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[4]')
            column3_texts.append(element_col3.text)
        except NoSuchElementException:
            column3_texts.append("NA")

        # 第四列 时间        
        try:
            element_col4 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[5]')
            column4_texts.append(element_col4.text)
        except NoSuchElementException:
            column4_texts.append("NA")



    # 尝试点击下一页按钮
    try:
        driver.find_element(By.ID, 'PageNext').click()
        print(f'正在爬起第{a}页')
        a += 1
        time.sleep(3)  # 等待页面加载 
    except NoSuchElementException:
        break  # 如果找不到下一页按钮，跳出循环 
    
df1=pd.DataFrame({
    '题名': column1_texts,
    '链接': column5_texts,
    '作者': column2_texts,
    '期刊名称': column3_texts,
    '时间': column4_texts,
    })


driver.find_element(By.ID, 'total').click() #点击首页
time.sleep(2)  # 等待页面加载

# 显示详细信息，为了爬取摘要、关键词
driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[2]/div/div[1]/div/div[2]/ul[2]/li[1]/i').click()
time.sleep(2)



#爬摘要和关键词
title_texts = []
abstract_texts = []
keywords_texts = []
a = 1 # 页码计数器

while True:
    elements_count = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/dl/dd'))
    if elements_count !=50:
        try:
            elements_count_contain = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/dl/font/dd'))
            elements_count = elements_count_contain + elements_count
            print(f"当前页{a}的1叠元素数量: {elements_count}")
            if elements_count != 50:
                elements_count_contain2 = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/dl/font/font/dd'))
                elements_count = elements_count_contain2 + elements_count
                print(f"当前页{a}的2叠元素数量: {elements_count}")
        except NoSuchElementException:
            elements_count = elements_count
            print(f"当前页{a}的元素数量: {elements_count}，不是50个，可能是最后一页")

    b=c=1
    for i in range(1, elements_count + 1):
        #题目路径
        xpath1=f'//*[@id="gridTable"]/div/div/div/dl/dd[{i}]/div[2]/h6/a'
        xpath2=f'//*[@id="gridTable"]/div/div/div/dl/font/dd[{b}]/div[2]/h6/a'
        xpath3=f'//*[@id="gridTable"]/div/div/div/dl/font/font/dd[{c}]/div[2]/h6/a'
        #摘要路径 
        xpath4=f'//*[@id="gridTable"]/div/div/div/dl/dd[{i}]/div[2]/p[2]/span[2]'
        xpath5=f'//*[@id="gridTable"]/div/div/div/dl/font/dd[{b}]/div[2]/p[2]/span[2]'
        xpath6=f'//*[@id="gridTable"]/div/div/div/dl/font/font/dd[{c}]/div[2]/p[2]/span[2]'
        #关键词路径 
        xpath7=f'//*[@id="gridTable"]/div/div/div/dl/dd[{i}]/div[2]/p[3]'
        xpath8=f'//*[@id="gridTable"]/div/div/div/dl/font/dd[{b}]/div[2]/p[3]'
        xpath9=f'//*[@id="gridTable"]/div/div/div/dl/font/font/dd[{c}]/div[2]/p[3]'
        #获取题目
        try:
            title_text = driver.find_element(By.XPATH, xpath1) 
            title_texts.append(title_text.text)
        except NoSuchElementException:
            try:
                title_text = driver.find_element(By.XPATH, xpath2) 
                title_texts.append(title_text.text)
                b+=1 
            except:
                try:
                    title_text = driver.find_element(By.XPATH,xpath3)
                    title_texts.append(title_text.text)
                    c+=1
                except:
                    title_texts.append("NA")
                    print(f"当前页{a}的元素数量: {elements_count}，第{i}个元素没有找到标题")
            
        #获取摘要
        try:
            abstract_text = driver.find_element(By.XPATH, xpath4) 
            abstract_texts.append(abstract_text.text)
        except NoSuchElementException:
            try:
                abstract_text = driver.find_element(By.XPATH, xpath5) 
                abstract_texts.append(abstract_text.text)
            except:
                try:
                    abstract_text = driver.find_element(By.XPATH,xpath6)
                    abstract_texts.append(abstract_text.text)
                except:
                    abstract_texts.append("NA")
                    print(f"当前页{a}的元素数量: {elements_count}，第{i}个元素没有找到摘要")
        
        #获取关键词
        try:
            keywords_text = driver.find_element(By.XPATH, xpath7) 
            keywords_texts.append(keywords_text.text)
        except NoSuchElementException:
            try:
                keywords_text = driver.find_element(By.XPATH, xpath8) 
                keywords_texts.append(keywords_text.text)
            except:
                try:
                    keywords_text = driver.find_element(By.XPATH,xpath9)
                    keywords_texts.append(keywords_text.text)
                except:
                    keywords_texts.append("NA")
                    print(f"当前页{a}的元素数量: {elements_count}，第{i}个元素没有找到关键词")
    print(f'正在爬第{a}页')
    
    # 尝试点击下一页按钮
    try:
        driver.find_element(By.ID, 'PageNext').click()
        a += 1
        time.sleep(3)  # 等待页面加载 
    except NoSuchElementException:
        break  # 如果找不到下一页按钮，跳出循环
keywords_texts = [text.replace("关键词：", "") for text in keywords_texts] # 去掉关键词前的“关键词：”

df2=pd.DataFrame({
    '题名': title_texts,
    '摘要': abstract_texts,
    '关键词': keywords_texts
})
driver.quit()# 关闭浏览器

#合并数据
df3 = pd.merge(df1, df2, on='题名', how='left')

#读期刊目录
excel_df = pd.read_excel(r'D:\东财外文期刊分类.xlsx')
excel_df = excel_df[['期刊名称', '级别']]

#为期刊配级
df4= pd.merge(df3, excel_df, on='期刊名称', how='left')



df4 = df4[['级别', '题名','链接', '作者', '期刊名称', '时间', '摘要', '关键词']]
df4['级别'] = df4['级别'].fillna('其他') # 将缺失值填充为'其他'




from datetime import datetime
today_date = datetime.now().strftime('%Y-%m-%d')
file_excel = f'{keyword}_外文文献标级_{today_date}.xlsx'
df4.to_excel(file_excel, index=False)

# 爬中文文献（不包含摘要关键词）

In [14]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException # 
import pandas as pd
import csv
import os


# 设置当前工作目录
os.chdir('D:\\漂绿文献\\爬虫\\文献爬虫数据') # 设置当前工作目录为你要保存文件的目录
keyword='社会责任报告语调'
# 初始化WebDriver
driver = webdriver.Edge()
url = 'https://www.cnki.net/'
# 知网高级检索链接
#url = 'https://kns.cnki.net/kns8s/AdvSearch?crossids=YSTTe4HG0%2CLSTPFY1C%2CJUP3MUPD%2CMPMFIG1A%2CWQ0UVIAA%2CBLZOG7CK%2CPWFIRAGL%2CEMRPGLPA%2CNLBO1Z6R%2CNN3FJMUV'
driver.get(url)
time.sleep(2)
#普通检索
driver.find_element(By.XPATH, '//*[@id="txt_SearchText"]').click()
driver.find_element(By.XPATH, '//*[@id="txt_SearchText"]').send_keys(keyword) 
driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[1]/input[2]').click() #点击检索
time.sleep(4)
# 高级检索
#driver.find_element(By.XPATH, '//*[@id="gradetxt"]/dd[1]/div[2]/input').click()

#driver.find_element(By.XPATH,'//*[@id="gradetxt"]/dd[1]/div[2]/input').send_keys(keyword) #检索词
#time.sleep(2)

# 进行检索//*  
#driver.find_element(By.XPATH,'//*[@id="gradetxt"]/dd[2]/a').click()                   
#driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[2]/div[3]/input').click()
time.sleep(4)
#cssci
#driver.find_element(By.XPATH,'//*[@id="divGroup"]/dl[2]/dd/div/ul/li[2]/input').click()
#cscd
#time.sleep(2)
#driver.find_element(By.XPATH,'//*[@id="divGroup"]/dl[2]/dd/div/ul/li[5]/input').click()



# 每页显示条数改成50
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/div/i').click()
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/ul/li[3]').click()

#只获取学术论文，排除学位论文、会议论文、报纸等
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[2]/div/div/ul/li[1]').click()

#time.sleep(2)
# 点击“学术论文”

# 创建空列表以存储所有元素的文本内容
column1_texts = []
column2_texts = []
column3_texts = []
column4_texts = []
column5_texts = []
column6_texts = []
column7_texts = []
a=1
# 循环遍历每一页
while True:
    # 获取当前页的元素数量  
    elements_count = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/table/tbody/tr'))
    if elements_count != 50:
        print(f"当前页{a}元素数量: {elements_count}，不是50，可能是最后一页")
    # 遍历当前页的元素，获取文本内容并存储到列表中
    for i in range(1, elements_count + 1):
        # 第一列 文章题目      //*[@id="gridTable"]/div/div/div/table/tbody/tr[1]/td[2]/a
        try:
            element = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[2]/a')
            column1_texts.append(element.text)
        except NoSuchElementException:
            column1_texts.append("NA")

        try:
            column7_texts.append(driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[2]/a').get_attribute('href'))
        except NoSuchElementException:
            column7_texts.append("NA")
        # 第二列 作者      
        try:
            #element_col2 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[3]')
            column2_texts.append(driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[3]').text)
        except NoSuchElementException:
            column2_texts.append("NA")

        # 第三列 期刊        
        try:
            #element_col3 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[4]/span/a')
            column3_texts.append(driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[4]/span/a').text)
        except NoSuchElementException:
            column3_texts.append("NA")

        # 第四列 时间        
        #try:
            #element_col4 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[5]')
            #column4_texts.append(driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[5]').text)
       # except NoSuchElementException:
           # column4_texts.append("NA")

      


    time.sleep(2)  # 等待页面加载
    # 尝试点击下一页按钮
    try:
        driver.find_element(By.ID, 'PageNext').click()
        print(f'正在爬一轮第{a}页')
        a += 1
        time.sleep(3)  # 等待页面加载 
    except NoSuchElementException:
        print('爬取结束')
        break  # 如果找不到下一页按钮，跳出循环 
driver.quit()# 关闭浏览器 
df1=pd.DataFrame({
    '题名': column1_texts,
    '链接': column7_texts,
    '作者': column2_texts,
    '期刊名称': column3_texts,
    '时间': column4_texts,
    })


#读期刊目录
excel_df = pd.read_excel(r'D:\东财中文期刊分类.xlsx')
excel_df = excel_df[['期刊名称', '级别']]

#为期刊配级
df4= pd.merge(df1, excel_df, on='期刊名称', how='left')
from datetime import datetime
df4 = df4[['级别', '题名','链接', '作者', '期刊名称', '时间']]
df4['级别'] = df4['级别'].fillna('其他') # 将缺失值填充为'其他'

today_date = datetime.now().strftime('%Y-%m-%d')
file_excel = f'{keyword}_中文文献标级_{today_date}.xlsx'
df4.to_excel(file_excel, index=False)

当前页1元素数量: 20，不是50，可能是最后一页


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: MicrosoftEdge=135.0.3179.73); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7465D7A55+24981]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF74652AC60+444240]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF74685E0EA+2011274]
	(No symbol) [0x00007FF746302430]
	(No symbol) [0x00007FF746305046]
	(No symbol) [0x00007FF746387343]
	(No symbol) [0x00007FF74636909A]
	(No symbol) [0x00007FF74633E13D]
	(No symbol) [0x00007FF7463867E8]
	(No symbol) [0x00007FF746368E03]
	(No symbol) [0x00007FF74633D686]
	(No symbol) [0x00007FF74633CB92]
	(No symbol) [0x00007FF74633D4B3]
	(No symbol) [0x00007FF74643900D]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF7464

# 爬外文文献（不包含摘要关键词）

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

import os

# 设置当前工作目录
os.chdir('D:\\漂绿文献\\爬虫\\文献爬虫数据') # 设置当前工作目录为你要保存文件的目录

# 初始化WebDriver
driver = webdriver.Edge()
url = 'https://kns.cnki.net/kns8s/AdvSearch?crossids=YSTT4HG0%2CLSTPFY1C%2CJUP3MUPD%2CMPMFIG1A%2CWQ0UVIAA%2CBLZOG7CK%2CPWFIRAGL%2CEMRPGLPA%2CNLBO1Z6R%2CNN3FJMUV'
driver.get(url)
time.sleep(2)

keyword='esg' 
# 点击知网检索输入栏，输入你的检索词
driver.find_element(By.XPATH, '//*[@id="gradetxt"]/dd[1]/div[2]/input').click()
driver.find_element(By.XPATH,'//*[@id="gradetxt"]/dd[1]/div[2]/input').send_keys(keyword) #检索词
time.sleep(3)

# 进行检索
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[2]/div[3]/input').click()
time.sleep(4)
# 每页显示条数改成50
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/div/i').click()
driver.find_element(By.XPATH,'//*[@id="perPageDiv"]/ul/li[3]/a').click()
time.sleep(3)


#只获取学术论文，排除学位论文、会议论文、报纸等
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[2]/div/div/ul/li[1]').click()
time.sleep(2)

# 点外文
driver.find_element(By.XPATH,'//*[@id="ModuleSearch"]/div[2]/div/div/div/div/a[2]').click()
time.sleep(2)

# 创建空列表以存储所有元素的文本内容
column1_texts = []
column2_texts = []
column3_texts = []
column4_texts = []
column5_texts = []

a=1
# 循环遍历每一页
while True:
    # 获取当前页的元素数量  //*[@id="gridTable"]/div/div/div/table/tbody
    elements_count = len(driver.find_elements(By.XPATH, '//*[@id="gridTable"]/div/div/div/table/tbody/tr'))
    if elements_count != 50:
        print(f"当前页{a}元素数量: {elements_count}，不是50，可能是最后一页")
    # 遍历当前页的元素，获取文本内容并存储到列表中
    for i in range(1, elements_count + 1):
        # 第一列 文章题目    
        try:
            element_col1 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[2]/a')
            column1_texts.append(element_col1.text)
            column5_texts.append(element_col1.get_attribute('href')) # 获取链接
        except NoSuchElementException:
            column1_texts.append("NA")

        # 第二列 作者      
        try:
            element_col2 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[3]')
            column2_texts.append(element_col2.text)
        except NoSuchElementException:
            column2_texts.append("NA")

        # 第三列 期刊        
        try:
            element_col3 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[4]')
            column3_texts.append(element_col3.text)
        except NoSuchElementException:
            column3_texts.append("NA")

        # 第四列 时间        
        try:
            element_col4 = driver.find_element(By.XPATH, f'//*[@id="gridTable"]/div/div/div/table/tbody/tr[{i}]/td[5]')
            column4_texts.append(element_col4.text)
        except NoSuchElementException:
            column4_texts.append("NA")



    # 尝试点击下一页按钮
    try:
        driver.find_element(By.ID, 'PageNext').click()
        print(f'正在爬起第{a}页')
        a += 1
        time.sleep(3)  # 等待页面加载 
    except NoSuchElementException:
        break  # 如果找不到下一页按钮，跳出循环 
    
df1=pd.DataFrame({
    '题名': column1_texts,
    '链接': column5_texts,
    '作者': column2_texts,
    '期刊名称': column3_texts,
    '时间': column4_texts,
    })
driver.quit()# 关闭浏览器
excel_df = pd.read_excel(r'D:\东财外文期刊分类.xlsx')
excel_df = excel_df[['期刊名称', '级别']]

#为期刊配级
df4= pd.merge(df1, excel_df, on='期刊名称', how='left')



df4 = df4[['级别', '题名','链接', '作者', '期刊名称', '时间']]
df4['级别'] = df4['级别'].fillna('其他') # 将缺失值填充为'其他'




from datetime import datetime
today_date = datetime.now().strftime('%Y-%m-%d')
file_excel = f'{keyword}_外文文献标级_{today_date}.xlsx'
df4.to_excel(file_excel, index=False)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=135.0.3179.73)
Stacktrace:
	GetHandleVerifier [0x00007FF7465D7A55+24981]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF74652AC60+444240]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF74685E0EA+2011274]
	(No symbol) [0x00007FF7462DC2B5]
	(No symbol) [0x00007FF74636EBF0]
	(No symbol) [0x00007FF7463861EA]
	(No symbol) [0x00007FF746368E03]
	(No symbol) [0x00007FF74633D686]
	(No symbol) [0x00007FF74633CB92]
	(No symbol) [0x00007FF74633D4B3]
	(No symbol) [0x00007FF74643900D]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF746446943+41539]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF74643FC79+13689]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7466BA04A+290794]
	Microsoft::Applications::Events::ILogConfiguration::operato

# Drissionpage 中文

In [2]:
from DrissionPage import ChromiumOptions, ChromiumPage
import pandas as pd
import time
import os

# 设置当前工作目录
os.chdir('D:\\漂绿文献\\爬虫') # 设置当前工作目录为你要保存文件的目录
# 打开网页
path=r'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'  # 请改为你电脑内Chrome可执行文件路径
co = ChromiumOptions().set_browser_path(path)
page = ChromiumPage(co)

keyword='社会责任披露'

page.get('https://www.cnki.net/')
page.ele('x://*[@id="txt_SearchText"]').input(keyword)
page.ele('x://*[@class="search-btn"]').click() #点击检索

page.ele('x://*[@id="perPageDiv"]/div/i').click() #点击每页显示条数
page.ele('x://*[@id="perPageDiv"]/ul/li[3]').click() #选择50条
page.ele('x://*[@id="ModuleSearch"]/div[2]/div/div/ul/li[1]').click() #只获取学术论文，排除学位论文、会议论文、报纸等


# 遍历当前页的元素，获取文本内容并存储到列表中 
titles, authors, journals, times, links = [], [], [], [], [] #创建空列表
a=0
while True:
   title=page.eles('x://*[@class="fz14"]') #获取题目
   author=page.eles('x://*[@class="author"]') #获取作者
   journal=page.eles('x://*[@class="source"]') #获取期刊
   time=page.eles('x://*[@class="date"]') #获取时间
   for i in zip(title, author, journal, time):
        titles.append(i[0].text) #获取题目文本
        authors.append(i[1].text) #获取作者文本     
        journals.append(i[2].text) #获取期刊文本
        times.append(i[3].text) #获取时间文本           
        links.append(i[0].attr('href')) #获取链接文本
   try:
        page.ele('x://*[@id="PageNext"]').click() #点击下一页
        a += 1 
        print(f'爬取第{a}页')
   except Exception as e:
        break

df1=pd.DataFrame({
    '题名': titles,
    '链接': links,
    '作者': authors,
    '期刊名称': journals,
    '时间': times
})


#读期刊目录
excel_df = pd.read_excel(r'D:\东财中文期刊分类.xlsx')
excel_df = excel_df[['期刊名称', '级别']]

#为期刊配级
df4= pd.merge(df1, excel_df, on='期刊名称', how='left')
from datetime import datetime
df4 = df4[['级别', '题名','链接', '作者', '期刊名称', '时间']]
df4['级别'] = df4['级别'].fillna('其他') # 将缺失值填充为'其他'

today_date = datetime.now().strftime('%Y-%m-%d')
file_excel = f'{keyword}_中文文献标级_{today_date}.xlsx'
df4.to_excel(file_excel, index=False)
page.quit()

爬取第1页
爬取第2页
爬取第3页
爬取第4页
爬取第5页
爬取第6页
爬取第7页
爬取第8页
爬取第9页
爬取第10页
爬取第11页
爬取第12页
爬取第13页
爬取第14页
爬取第15页
爬取第16页
爬取第17页
爬取第18页
爬取第19页
爬取第20页
爬取第21页
爬取第22页
爬取第23页
爬取第24页
爬取第25页
爬取第26页
爬取第27页
爬取第28页
爬取第29页
爬取第30页
爬取第31页
爬取第32页
爬取第33页
爬取第34页
爬取第35页
爬取第36页
爬取第37页
爬取第38页
爬取第39页
爬取第40页
爬取第41页
爬取第42页
爬取第43页


# 外文

In [3]:

from DrissionPage import ChromiumOptions, ChromiumPage
import pandas as pd
import time
import os

# 设置当前工作目录
os.chdir('D:\\漂绿文献\\爬虫\\文献爬虫数据') # 设置当前工作目录为你要保存文件的目录
# 打开网页
path=r'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'  # 请改为你电脑内Chrome可执行文件路径
co = ChromiumOptions().set_browser_path(path)
page = ChromiumPage(co)


keyword='esg语调'

page.get('https://www.cnki.net/')
page.ele('x://*[@id="txt_SearchText"]').input(keyword)
page.ele('x://*[@class="search-btn"]').click() #点击检索
time.sleep(4)
#外文
page.ele('x://*[@id="ModuleSearch"]/div[2]/div/div/div/div/a[2]').click() #点击外文
time.sleep(1)
page.ele('x://*[@id="perPageDiv"]/div').click() #点击每页显示条数
page.ele('x://*[@id="perPageDiv"]/ul/li[3]').click() #选择50条
page.ele('x://*[@id="ModuleSearch"]/div[2]/div/div/ul/li[1]').click() #只获取学术论文，排除学位论文、会议论文、报纸等


         
time.sleep(2)

# 遍历当前页的元素，获取文本内容并存储到列表中 
titles, authors, journals, times, links = [], [], [], [], [] #创建空列表
a=0
while True:
   title=page.eles('x://*[@class="fz14"]') #获取题目
   author=page.eles('x://*[@class="author"]') #获取作者
   journal=page.eles('x://*[@class="source"]') #获取期刊
   time=page.eles('x://*[@class="date"]') #获取时间
   for i in zip(title, author, journal, time):
        titles.append(i[0].text) #获取题目文本
        authors.append(i[1].text) #获取作者文本     
        journals.append(i[2].text) #获取期刊文本
        times.append(i[3].text) #获取时间文本           
        links.append(i[0].attr('href')) #获取链接文本
   try:
        page.ele('x://*[@id="PageNext"]').click() #点击下一页
        a += 1 
        print(f'爬取第{a}页')
   except Exception as e:
        break

import pandas as pd
df1=pd.DataFrame({
    '题名': titles,
    '链接': links,
    '作者': authors,
    '期刊名称': journals,
    '时间': times
})


#读期刊目录
excel_df = pd.read_excel(r'D:\东财外文期刊分类.xlsx')
excel_df = excel_df[['期刊名称', '级别']]

#为期刊配级
df4= pd.merge(df1, excel_df, on='期刊名称', how='left')
from datetime import datetime
df4 = df4[['级别', '题名','链接', '作者', '期刊名称', '时间']]
df4['级别'] = df4['级别'].fillna('其他') # 将缺失值填充为'其他'
#删除重复值
df4 = df4.drop_duplicates(subset=['题名', '链接', '作者', '期刊名称', '时间'], keep='first').reset_index(drop=True)

today_date = datetime.now().strftime('%Y-%m-%d')
file_excel = f'{keyword}_英文文献标级_{today_date}.xlsx'
df4.to_excel(file_excel, index=False)
page.quit() # 关闭浏览器

NoRectError: 
该元素没有位置及大小。
版本: 4.1.0.18

# 翻译

In [6]:
from translate import Translator
import pandas as pd
file_excel= r'D:\漂绿文献\爬虫\文献爬虫数据\漂绿_英文文献标级_2025-07-14.xlsx'  # 请改为你要翻译的Excel文件路径
df = pd.read_excel(file_excel)

translator = Translator(to_lang="zh")
title_chineses = []
a = 1
for i in df['题名']:
    title_chinese = translator.translate(i)
    title_chineses.append(title_chinese)
    print(f"正在翻译第{a}个标题，翻译为：{title_chinese}")
    a += 1

df['中文题名'] = title_chineses
df.to_excel(f'{file_excel[:-5]}_翻译版.xlsx', index=False)

正在翻译第1个标题，翻译为：MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  06 HOURS 38 MINUTES 23 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE
正在翻译第2个标题，翻译为：MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  06 HOURS 38 MINUTES 22 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE


KeyboardInterrupt: 